In [37]:
import numpy as np
from numpy.linalg import norm
import matplotlib.pyplot as plt
import scipy.integrate
from matplotlib.pyplot import figure

In [38]:
import sys
np.set_printoptions(threshold=sys.maxsize)

### Problem 1 

In [39]:
# Define the function
def rhsfunc(x, y, ep):
    f1 = y[1]
    f2 = (x**2 - ep) * y[0]
    
    return np.array([f1,f2])

In [40]:
# Define some constants 
K = 1
L = 4
ep_start = 0
tol = 10**(-6) 

xp = [-L, L]
x_evals = np.linspace(-L,L,20*L+1) 

In [41]:
ep = ep_start
dep = K/100 

In [42]:
# Define our initial conditions 
y0 = np.array([1,((K*L**2)-ep)**.5])

In [43]:
A6 = np.array([])
A = {}

for modes in range(5):
    ep = ep_start
    dep = K/100
    
    
    for j in range(1000): # using for loop to make sure it stops. 
        sol = scipy.integrate.solve_ivp(lambda x,y: rhsfunc(x,y,ep), xp, y0, t_eval = x_evals)
        y_sol = sol.y[0,:] # f1 
        y_sol_dx = sol.y[1,:] #f2 
        
        # Normalize eigenfunction
        y_sol_norm = y_sol/(np.sqrt(scipy.integrate.trapz(y_sol**2, x_evals)))

        if np.abs(y_sol_dx[-1] + (((K*L**2)-ep)**.5)*y_sol[-1]) < tol:
            print('We got the eigenvalue:', ep)
            A6 = np.append(A6, ep)
            break 

        if (-1)**(modes)*(y_sol_dx[-1] + (((K*L**2)-ep)**.5)*y_sol[-1]) > tol:
            ep = ep + dep # Increase 
        else: 
            ep = ep - dep/2 # Decrease 
            dep = dep/2 # Cut dep in half to make sure we converge 
    
    A[modes] = y_sol_norm
    
    ep_start = ep + 0.1
    
#     print(y_sol_norm)
#     plt.plot(sol.t, y_sol_norm, linewidth=2)
#     plt.plot(sol.t, sol.y[0, :], linewidth=2)
#     plt.plot(sol.t, 0*sol.t, 'k')

In [44]:
A1 = np.abs(A[0].copy()).reshape(81,1)
A2 = np.abs(A[1].copy()).reshape(81,1)
A3 = np.abs(A[2].copy()).reshape(81,1)
A4 = np.abs(A[3].copy()).reshape(81,1)
A5 = np.abs(A[4].copy()).reshape(81,1)

In [45]:
A6 = A6.reshape(1,5)

In [ ]:
eigenfuncs = np.vstack((A[0],A[1],A[2],A[3],A[4]))
eigenfuncs = eigenfuncs.T
eigenfuncs.shape

In [ ]:
L = 3
x = np.linspace(-L, L, 20*L+1)
fig, ax = plt.subplots()
ax.plot(x, eigenfuncs, linewidth=3)
ax.legend([r"$\phi_1$", r"$\phi_2$", r"$\phi_3$", r"$\phi_4$", r"$\phi_5$"])

In [ ]:
eigenfuncs[0,:]

In [ ]:
f = np.cos(x**2)

In [ ]:
a = np.zeros([1, len(eigenfuncs[0,:])])
for j in range(len(eigenfuncs[0,:])):
    a[:,j] = np.trapz( eigenfuncs[:, j]*f, x = x)

In [ ]:
t = np.linspace(0, 5, 100);

In [ ]:
part = j*np.outer(eigenvals, t)/2

### Problem 3

In [66]:
# Define the function
def rhs_ivp(x, y, ep, gamma):
    f1 = y[1]
    f2 = ((gamma*np.abs(y[0])**2)+x**2 - ep) * y[0]
    
    return np.array([f1,f2])

In [67]:
# Define some constants 
K = 1
L = 3
tol = 10**(-5) 

xp = [-L, L]
x_span = np.linspace(-L,L,20*L+1) 
gamma = [0.05, -0.05]

In [68]:
A15_18 = np.array([])
Eigenfunctions_all = np.array([])

for gamma in gamma:
    A = 0.000001
    ep_start = 0j
    ep = ep_start
    dep = K/100 
    y0 = np.array([A,(((K*L**2) - ep)**.5)*(A)])

    for modes in range(2):
        ep = ep_start
        dep = K/100

        for j in range(1000): # using for loop to make sure it stops. 
            # update initial condition
            y0 = np.array([A,((((K*L**2) - ep))**.5)*(A)])

            # solve ODE 
            sol = scipy.integrate.solve_ivp(lambda x,y: rhs_ivp(x,y,ep,gamma), xp, y0, t_eval = x_span)
            y_sol = sol.y[0,:] # f1 
            y_sol_dx = sol.y[1,:] #f2

            # define/compute norm and boundary conditions 
            norm = np.abs(scipy.integrate.trapz(y_sol**2,x_span)) #norm
            norm_tol = np.abs(scipy.integrate.trapz(y_sol**2,x_span)-1)
            boundary = np.abs(y_sol_dx[-1] + (((K*L**2)-ep)**.5)*y_sol[-1]) # same bc as problem 1 and 2. 
    #         boundary = np.abs(y_sol_dx[-1] + (((gamma*np.abs(y_sol[-1])**2)+(K*L**2)-ep)**.5)*y_sol[-1])

    # Norm Shooting 

            #if norm and boundary condiiton met, break, else A = A/sqrt(norm)
            if boundary < tol and norm_tol < tol:
                break
            else:
                A = A/np.sqrt(norm)
    #             print('norm:', norm)
    #             print('A:', A)

    # BC Shooting

            # update initial condition with new A 
            y0 = np.array([A,((((K*L**2) - ep))**.5)*(A)])

            # solve ODE 
            sol = scipy.integrate.solve_ivp(lambda x,y: rhs_ivp(x,y,ep,gamma), xp, y0, t_eval = x_span)
            y_sol = sol.y[0,:] # f1 
            y_sol_dx = sol.y[1,:] #f2 

            # define/compute norm and boundary conditions 
            norm = np.abs(scipy.integrate.trapz(y_sol**2,x_span)) #norm
            norm_tol = np.abs(scipy.integrate.trapz(y_sol**2,x_span)-1)
            boundary = np.abs(y_sol_dx[-1] + (((K*L**2)-ep)**.5)*y_sol[-1])

            if boundary < tol and norm_tol < tol:
                break

            if (-1)**(modes)*(y_sol_dx[-1] + (((K*L**2)-ep)**.5)*y_sol[-1]) > tol:
                ep = ep + dep # Increase 
            else:
                ep = ep - dep/2 # Decrease 
                dep = dep/2 # Cut dep in half to make sure we converge

        ep_start = ep + 0.1 
        A = A

        A15_18 = np.append(A15_18, ep)
        Eigenfunctions_all = np.append(Eigenfunctions_all, y_sol)

In [69]:
A15 = A15_18[0:2].reshape(1,2)
A18 = A15_18[2:4].reshape(1,2)

# print('eigenvalues for gamma = 0.05', A15)
# print('eigenvalues for gamma = 0', A15_18[4:46].reshape(1,2))
# print('eigenvalues for gamma = -0.05', A18)

In [70]:
A13 = np.abs(Eigenfunctions_all[0:61].reshape(61,1))

In [71]:
A14 = np.abs(Eigenfunctions_all[61:122].reshape(61,1))

In [72]:
A16 = np.abs(Eigenfunctions_all[122:183].reshape(61,1))

In [73]:
A17 = np.abs(Eigenfunctions_all[183:244].reshape(61,1))